In [6]:
import pandas as pd, numpy as np
import requests, re, locale
from bs4 import BeautifulSoup

In [7]:
def date_parser(date):
    f = lambda x: re.search(r'(([0-9]|[1-3][0-9]) de [a-z]{4,10} de (19|20)\d{2})', x)
    try:
        return datetime.datetime.strptime(date, r'%d de %B de %Y')
    except ValueError:
        return date_parser(f(date).group(0))
    except:
        print('Error.')

In [8]:
def scrap(url, attrs = {'class': 'remate'}):
    """
    Devuelve un df con datos de una web
    url: str - dirección web
    
    """
    #url_rosgan = r'http://rosgan.com.ar/precios-indices-rosgan/'
    r = requests.get(url = url)
    soup = BeautifulSoup(r.text, 'lxml')
    data = soup.find_all('dl', class_ = 'remate')
    #soup = soup.find_all(attrs = attrs)
    data = pd.DataFrame([(e.dt.text, e.dd.div.span.text) for e in data])
    #data[1].str.extract(pat = r'([0-9]{1,3}.[0-9])').astype(float)
    #data[['nro', 'fecha']] = data[0].str.split('-', expand = True)
    data = data.agg({1 : lambda x: x.str.extract(pat = r'([0-9]{1,3}.[0-9])').astype(float),
                     0 : lambda x: x.str.split('-', expand = True)})
    data.columns = ['precio', 'titulo', 'fecha']
    data['fecha'] = data.agg({'fecha' : date_parser})
    return data.set_index('fecha').sort_index()

In [9]:
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

'es_ES.UTF-8'

In [13]:
# Carga de datos precio rosgan y resampleo mensual OJO
#url_rosgan = r'http://rosgan.com.ar/precios-indices-rosgan/'
url_rosgan = r'https://www.rosgan.com.ar/precios-rosgan/'
rosgan_precio = scrap(url = url_rosgan)
#rosgan_precio = pd.read_csv('csv/rosgan.csv', parse_dates = [0], index_col = [0])
#rosgan_precio = rosgan_precio.resample('1m').last().interpolate().to_period('1m')
rosgan_precio

Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.

,precio,titulo
fecha,,
NaN,341.9,Especial Expoagro
NaN,283.3,Rosgan Nro.154
NaN,272.0,Rosgan Nro.153
NaN,289.8,Rosgan Nro.152
NaN,262.6,Rosgan Nro.151
...,...,...
NaN,6.3,Remate Nro 15
NaN,4.5,Remate Nro 14
NaN,3.8,Remate Nro 13


In [14]:
rosgan_precio

,precio,titulo
fecha,,
NaN,341.9,Especial Expoagro
NaN,283.3,Rosgan Nro.154
NaN,272.0,Rosgan Nro.153
NaN,289.8,Rosgan Nro.152
NaN,262.6,Rosgan Nro.151
...,...,...
NaN,6.3,Remate Nro 15
NaN,4.5,Remate Nro 14
NaN,3.8,Remate Nro 13
